In [ ]:
import os
import re
import time
import datetime
from tqdm import tqdm 
import pandas as pd
import requests

In [ ]:
import sys
from pathlib import Path
project_root = Path.cwd()  # 找出根目錄：Path.cwd()找出現在所在目錄(/run).parent(上一層是notebook).parent(再上層一層business_district_discovery)
print(project_root)
sys.path.append(str(project_root))

from config import plvrurls_1140416, plvr_column_names
from utils import combined_df, parse_admin_region

In [ ]:
import_name = 'plvrurls_1140416'
input_time = import_name.split('_')[1]  # 提取 "1140416" 部分

df = combined_df(plvrurls_1140416, input_time)

In [ ]:
if not os.path.exists('../data'):
    os.makedirs('../data')
    
# 使用今天的日期作為檔名的一部分
today = datetime.datetime.now().strftime('%Y%m%d')
df.to_csv(f'../data/plvr_rawdata_{today}.csv', index=False, encoding='utf-8-sig')
df.to_pickle(f'../data/plvr_rawdata_{today}.pkl')

In [ ]:
# df = pd.read_csv('../data/plvr_rawdata_20250417.csv', encoding='utf-8-sig', dtype={'AA12': 'str', 'cp': 'str', 's': 'str'})
df = pd.read_pickle('../data/plvr_rawdata_20250417.pkl')

In [ ]:
df = df.rename(columns=plvr_column_names, inplace=False)

In [ ]:
proc_df = df.copy()
proc_df = proc_df[["縣市", "坐落街道", "建物型態", "社區名稱", "棟號", "交易日期", "總面積", "交易總價",  
"建物單價", "樓層", "解約情形", "備查編號", "備註", "主要用途",  "車位總價", "車位筆數",  "經度", "緯度"]]

In [ ]:
# 在 proc_df 裡面新增一個「行政區」欄位
proc_df["行政區"] = proc_df["坐落街道"].apply(parse_admin_region)
# 新增「交易年月」欄位
proc_df['交易年月'] = proc_df['交易日期'].str.replace(r'(\d+)/(\d+)/\d+', r'\1\2', regex=True)

# 「交易總價」及「交易單價」由字串轉為數值
proc_df['交易總價'] = pd.to_numeric(proc_df['交易總價'].str.replace(',', ''), errors='coerce')
proc_df['交易總價'] = (proc_df['交易總價'] / 10000).round(0).astype(int)

proc_df

In [ ]:
proc_df['建物單價'] = pd.to_numeric(proc_df['建物單價'].str.replace(',', ''), errors='coerce')
proc_df['建物單價'] = (proc_df['建物單價'] / 10000).round(1).fillna(0).astype(float)
proc_df

In [ ]:
final_df = proc_df[["縣市", "行政區", "坐落街道", "建物型態", "社區名稱", "棟號", "交易日期", "總面積", "交易總價",  
"建物單價", "樓層", "解約情形", "備查編號", "備註", "主要用途",  "車位總價", "車位筆數",  "經度", "緯度"]]
final_df.to_pickle('../data/plvr_output_20250417.pkl')